In [ ]:
# @title ###### Licensed to the Apache Software Foundation (ASF), Version 2.0 (the "License")

# Licensed to the Apache Software Foundation (ASF) under one
# or more contributor license agreements. See the NOTICE file
# distributed with this work for additional information
# regarding copyright ownership. The ASF licenses this file
# to you under the Apache License, Version 2.0 (the
# "License"); you may not use this file except in compliance
# with the License. You may obtain a copy of the License at
#
#   http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing,
# software distributed under the License is distributed on an
# "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY
# KIND, either express or implied. See the License for the
# specific language governing permissions and limitations
# under the License

# Apache Beam RunInference with TensorFlow

<table align="left">
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/apache/beam/blob/master/examples/notebooks/beam-ml/run_inference_tensorflow.ipynb"><img src="https://raw.githubusercontent.com/google/or-tools/main/tools/colab_32px.png" />Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/apache/beam/blob/master/examples/notebooks/beam-ml/run_inference_tensorflow.ipynb"><img src="https://raw.githubusercontent.com/google/or-tools/main/tools/github_32px.png" />View source on GitHub</a>
  </td>
</table>


This notebook demonstrates the use of the [RunInference](https://beam.apache.org/releases/pydoc/current/apache_beam.ml.inference.base.html#apache_beam.ml.inference.base.RunInference) transform for [TensorFlow](https://www.tensorflow.org/).

Beam has built in support for 2 TensorFlow Model Handlers: [TFModelHandlerNumpy](https://github.com/apache/beam/blob/ca0787642a6b3804a742326147281c99ae8d08d2/sdks/python/apache_beam/ml/inference/tensorflow_inference.py#L91) and [TFModelHandlerTensor](https://github.com/apache/beam/blob/ca0787642a6b3804a742326147281c99ae8d08d2/sdks/python/apache_beam/ml/inference/tensorflow_inference.py#L184).
TFModelHandlerNumpy can be used to run inference on models expecting a `numpy` array as an input while TFModelHandlerTensor can be used to run inference on models expecting a `tf.Tensor` as an input.

If your model needs input of type `tf.Example` see the [Apache Beam RunInference with `tfx-bsl` notebook](https://colab.research.google.com/github/apache/beam/blob/master/examples/notebooks/beam-ml/run_inference_tensorflow_with_tfx.ipynb).

The Apache Beam RunInference transform is used to make predictions for
a variety of machine learning models. For more information about the RunInference API, see [Get started with AI/ML pipelines](https://beam.apache.org/documentation/ml/overview/) in the Apache Beam documentation.

This notebook demonstrates the following steps:
- Build a simple TensorFlow model.
- Set up example data.
- Run those examples with the built-in model handlers and get a prediction inside an Apache Beam pipeline.

To use RunInference with built-in Tensorflow model handler, install Apache Beam version 2.46 or later.

In [ ]:
!pip install protobuf --quiet
!pip install apache_beam==2.46.0 --quiet

### Authenticate with Google Cloud
This notebook relies on saving your model to Google Cloud. To use your Google Cloud account, authenticate this notebook.

In [2]:
from google.colab import auth
auth.authenticate_user()

### Import dependencies and set up your bucket
Replace `PROJECT_ID` and `BUCKET_NAME` with the ID of your project and the name of your bucket.

**Important**: If an error occurs, restart your runtime.

In [10]:
import argparse
from typing import Dict, Text, Any, Tuple, List
import numpy

from google.protobuf import text_format

import tensorflow as tf
from tensorflow import keras
import apache_beam as beam
from apache_beam.ml.inference.base import RunInference
from apache_beam.ml.inference.base import KeyedModelHandler
from apache_beam.ml.inference.tensorflow_inference import TFModelHandlerNumpy
from apache_beam.ml.inference.tensorflow_inference import TFModelHandlerTensor
from apache_beam.options.pipeline_options import PipelineOptions

project = "PROJECT_ID"
bucket = "BUCKET_NAME"

save_model_dir_multiply = f'gs://{bucket}/tfx-inference/model/multiply_five/v1/'


## Create and test a simple model

This step creates and tests a model that predicts the 5 times table.

### Create the model
Create training data and build a linear regression model.

In [6]:
# Create training data that represents the 5 times multiplication table for the numbers 0 to 99.
# x is the data and y is the labels.
x = numpy.arange(0, 100)   # Examples
y = x * 5                  # Labels

# Build a simple linear regression model.
# Note that the model has a shape of (1) for its input layer and expects a single int64 value.
input_layer = keras.layers.Input(shape=(1), dtype=tf.float32, name='x')
output_layer= keras.layers.Dense(1)(input_layer)

model = keras.Model(input_layer, output_layer)
model.compile(optimizer=tf.optimizers.Adam(), loss='mean_absolute_error')
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 x (InputLayer)              [(None, 1)]               0         
                                                                 
 dense (Dense)               (None, 1)                 2         
                                                                 
Total params: 2
Trainable params: 2
Non-trainable params: 0
_________________________________________________________________


### Test the model

This step tests the model that you created.

In [7]:
model.fit(x, y, epochs=500, verbose=0)
test_examples =[20, 40, 60, 90]
value_to_predict = numpy.array(test_examples, dtype=numpy.float32)
predictions = model.predict(value_to_predict)

print('Test Examples ' + str(test_examples))
print('Predictions ' + str(predictions))

1/1 [==============================] - 0s 64ms/step
Test Examples [20, 40, 60, 90]
Predictions [[ 51.815357]
 [101.63492 ]
 [151.45448 ]
 [226.18384 ]]


### Save the model

This step shows how to save your model.

In [8]:
model.save(save_model_dir_multiply)

## Run the Pipeline
Use the following code to run the pipeline.

In [9]:
class FormatOutput(beam.DoFn):
  def process(self, element, *args, **kwargs):
     yield "example is {example} prediction is {prediction}".format(example=element.example, prediction=element.inference)


examples = numpy.array([20, 40, 60, 90], dtype=numpy.float32)
model_handler = TFModelHandlerNumpy(save_model_dir_multiply)
with beam.Pipeline() as p:
    _ = (p | beam.Create(examples)
           | RunInference(model_handler)
           | beam.ParDo(FormatOutput())
           | beam.Map(print)
        )

example is 20.0 prediction is [51.815357]
example is 40.0 prediction is [101.63492]
example is 60.0 prediction is [151.45448]
example is 90.0 prediction is [226.18384]


## KeyedModelHandler with TensorFlow using TFModelHandlerNumpy

By default, the `ModelHandler` does not expect a key.

* If you know that keys are associated with your examples, wrap the model handler with `beam.KeyedModelHandler`.
* If you don't know whether keys are associated with your examples, use `beam.MaybeKeyedModelHandler`.

In [11]:
class FormatOutputKeyed(FormatOutput):
  # To simplify, inherit from FormatOutput.
  def process(self, tuple_in: Tuple):
    key, element = tuple_in
    output = super().process(element)
    yield "{} : {}".format(key, [op for op in output])

examples = numpy.array([(1,20), (2,40), (3,60), (4,90)], dtype=numpy.float32)
keyed_model_handler = KeyedModelHandler(TFModelHandlerNumpy(save_model_dir_multiply))
with beam.Pipeline() as p:
    _ = (p | 'CreateExamples' >> beam.Create(examples)
           | RunInference(keyed_model_handler)
           | beam.ParDo(FormatOutputKeyed())
           | beam.Map(print)
        )

1.0 : ['example is 20.0 prediction is [51.815357]']
2.0 : ['example is 40.0 prediction is [101.63492]']
3.0 : ['example is 60.0 prediction is [151.45448]']
4.0 : ['example is 90.0 prediction is [226.18384]']
